In [0]:
silver_table = "dataanalysispysparkbook.baby_names_clean"
gold_table_path = "/Volumes/workspace/dataanalysispysparkbook/gold/baby_names_summary"
gold_table = "dataanalysispysparkbook.baby_names_summary"

# Agregación
df_summary = spark.sql(f"""
        SELECT Year, Name, SUM(Count) AS total
            FROM {silver_table}
        GROUP BY Year, Name
        ORDER BY Year, total DESC
        """)
df_summary.limit(5).display()

# Guardar como Delta en Gold
df_summary.write.format("delta").mode("overwrite").save(gold_table_path)

# Crear vista temporal (corregido)
df_summary.createOrReplaceTempView("v_baby_names_summary")

# Registrar tabla
spark.sql(f"""CREATE TABLE IF NOT EXISTS {gold_table} AS SELECT * FROM v_baby_names_summary""")
spark.sql(f'''SELECT * FROM {gold_table} LIMIT 5''').display()